In [1]:
%run Utils.ipynb
from sage.rings.finite_rings.integer_mod import IntegerMod_gmp

In [10]:
class Verifier_first_stage():
    
    def __init__(self, Y:IntegerMod_gmp , Y_index : int, excecution_trace_length: int, num_of_queries: int):
        self.domain = get_extented_domain(excecution_trace_length=excecution_trace_length, num_of_queries=num_of_queries)
        self.domian_size = excecution_trace_length*num_of_queries
        self.Y_index = Y_index
        self.Y = Y
        self.num_of_queries = num_of_queries
    
    def verify(self, proof_stage_one: dict, proof_stage_two: dict):
            indexes = self.get_indexes(proof_stage_one=proof_stage_one, proof_stage_two=proof_stage_two)
            coeffs = self.get_coefficients(proof_stage_one=proof_stage_one)
            
            for index in indexes:
                self.verifiy_path_and_values(proof_stage_one=proof_stage_one, index=index)
                ps_dict = self.calculate_ps_by_index(proof_stage_one=proof_stage_one, index=index)
                # self.validate_samples(proof_stage_one=proof_stage_one, index=index, ps_dict=ps_dict)
                self.validate_cp_value_in_index(proof_stage_one=proof_stage_one, coeffs=coeffs, ps_dict=ps_dict, index=index)

    def verifiy_path_and_values(self, proof_stage_one: dict, index: int):
        names = ['ex_poly', 'cp']
        for name in names:
            internal_proof_dict = proof_stage_one[name]
            root = internal_proof_dict['root']
            value_at_index = internal_proof_dict['value'][index]
            path = internal_proof_dict['path'][index]
            verify_path_by_index(
                root=root, expected_value=value_at_index, index=index, domain_size=self.domian_size, path=path
            )

    def get_coefficients(self, proof_stage_one:dict) -> dict:
        c1, c2, c3 = create_poly_coeff(proof_stage_one['ex_poly']['root'])
        return ({'p1':c1, 'p2':c2, 'p3':c3})
        #return {name:fiat_shamir_random(proof_stage_one[name]['root']) for name in names}
    
    def validate_cp_value_in_index(self, proof_stage_one: dict, coeffs: dict,  ps_dict: dict, index: int):
        names = ['p1', 'p2', 'p3']
        expected_value = 0
        for name in names:
            expected_value_int = ps_dict[name][1]*coeffs[name]
            expected_value += expected_value_int%PRIME    
        print(f"{proof_stage_one['cp']['value'][index][1]=}  {expected_value=}")
        assert proof_stage_one['cp']['value'][index][1] == expected_value
        
        
    def validate_samples(self, proof_stage_one: dict, index: int, ps_dict):
        
        names = ['p1', 'p2', 'p3']
        for name in names:
            (x0, y0) = (proof_stage_one[name]['value'][index])
            print(f"{(x0, y0)=}   {ps_dict[name]=}")
            assert self.domain[index] == x0
            assert ps_dict[name] == (x0, y0)
    
    def calculate_ps_by_index(self, proof_stage_one: dict, index: int) -> dict:
        #Create a copy of poly(x), poly(g*x), poly(g**2 *x)
    
        exceqution_polinomial_samples=proof_stage_one['ex_poly_samples']           
        p_x, p_x_g, p_x_g_2 = self.get_mask_by_index(proof_stage_one=proof_stage_one, index=index)
        ps_dict = self.calculate_constrais_polynomials_samples_by_index(index, p_x, p_x_g, p_x_g_2)
        return ps_dict
        

    def get_mask_by_index(self, proof_stage_one: dict, index: int) -> list:
        exceqution_polinomial_samples=proof_stage_one['ex_poly_samples']
        p_x = exceqution_polinomial_samples['ex_p()'][index]
        p_x_g = exceqution_polinomial_samples['ex_p(g)'][index]
        p_x_g_2 = exceqution_polinomial_samples['ex_p(g**2)'][index]
        return (p_x, p_x_g, p_x_g_2)

    def calculate_constrais_polynomials_samples_by_index(
        self, index: int, p_x: IntegerMod_gmp, p_x_g: IntegerMod_gmp, p_x_g_2: IntegerMod_gmp
    ) -> dict:
            # Validate_p1_value:
            x1 = self.domain[index]
            y1 = (p_x-1)._divide_if_possible(x1-gen1024**0)
            p1=(x1, y1)
            # p1 = (poly-1)._divide_if_possible(x-gen**0)
        
            # Validate_p2_value:
            x2 = self.domain[index]
            y2 = (p_x-self.Y)._divide_if_possible(x2-gen1024**self.Y_index)
            p2 = (x2, y2)


            # Validate_p3_value:
            x3 = self.domain[index]
            n=1024 #trance_len
            constrain_3_numer = p_x_g_2 - p_x_g - p_x
            constrain_3_denom = (x3**n-1)._divide_if_possible( (x3-gen1024**(n-1)) * (x3-gen1024**(n-2)))
            y3 = constrain_3_numer._divide_if_possible(constrain_3_denom)
            p3 = (x3, y3)
            
            return({'p1':p1, 'p2':p2, 'p3':p3})
        
    
    def get_indexes(self, proof_stage_one: dict, proof_stage_two: dict):
        return get_random_index_to_sample(
            queries=self.num_of_queries, merkel_root=proof_stage_two['root'], domain_len=self.domian_size
        )

In [2]:
def verify_second_stage(proof_stage_two: dict, root_stage_zero: str, degree: int, queries: int, field_gen = field_gen): 
    domain_len = queries * degree
    roots = proof_stage_two['root']
    indexes = get_random_index_to_sample(queries, roots, domain_len)
    stages = log(degree, 2)
    pathes = proof_stage_two['pathes']
    #value at index = pathes[stage][index][0]
    #path of value at index = pathes[stage][index][1]
    
    #veify stages number:
    assert stages == len(roots)
    
    for stage in range(stages):
        #verify all point's pathes according to the root
        for index in indexes:
            expected_value = pathes[stage][index][0]
            path = pathes[stage][index][1]
            verify_path_by_index(roots[stage], expected_value, index , domain_len, path)
        #verify first stage points and calculate next stage points
        current_stage_points = [pathes[stage][index][0] for index in indexes]
        rand = fiat_shamir_random(roots[stage])
        if stage==0:
            assert roots[stage] == root_stage_zero
            verify_index_x_value(current_stage_points, indexes, domain_len, field_gen)
            next_stage_points = compute_first_layer(current_stage_points, rand)
        else:
            next_stage_points = compute_next_layer(current_stage_points, calculated_points, rand)
        
        calculated_points =  next_stage_points
        if stage + 1 in range(stages):
            indexes = get_next_stage_indexes(indexes)
        domain_len /= 2
    #verify that the last layer is constant
    for i in range(len(calculated_points)-1):
        assert calculated_points[i][1] == calculated_points[i+1][1]
    
    return True

In [ ]:
def compute_first_layer(current_stage_point, rand):
    next_stage_points =[]
    for i in range(0,len(current_stage_point),2):
        a,f_a = current_stage_point[i]
        neg_a, neg_f_a = current_stage_point[i+1]
        y = f_a + (f_a - neg_f_a)/(2*a)*(rand-a)
        next_stage_points.append((a**2,y))
    return next_stage_points
        

def compute_next_layer(current_stage_point, calculated_points, rand):
    next_stage_points = []
    for point_a, point_neg_a in zip (current_stage_point, calculated_points):
        a,f_a = point_a
        neg_a, neg_f_a = point_neg_a
        y = f_a + (f_a - neg_f_a)/(2*a)*(rand-a)
        next_stage_points.append((a**2,y))
    return next_stage_points

def verify_index_x_value(current_stage_points, indexes, domain_len, field_gen):
    group_gen = field_gen ** ((PRIME-1)/domain_len)
    new_indexes = [reverse_bit(i, log(domain_len,2)) for i in indexes]
    alleged_x = [field_gen*(group_gen**i) for i in new_indexes]
    prover_x = [x[0] for x in current_stage_points]
    assert alleged_x == prover_x